In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Hp\\Desktop\\Kidney-Disease-classification-DVC-MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Hp\\Desktop\\Kidney-Disease-classification-DVC-MLflow'

In [5]:
import tensorflow as tf

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from kidney_desiese_classifier.constants import *
from kidney_desiese_classifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_file_path,
        params_filepath = params_file_path):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/unzip_data",
            all_params=self.params,
            params_image_size=self.params.Image_size,
            params_batch_size=self.params.Batch_size
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
from urllib.parse import urlparse

In [23]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config
        
        
    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator() 
        self.score = self.model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        print(scores)

In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    eval = Evaluation(eval_config)
    eval.evaluation()
    eval.save_score()
except Exception as e:
    raise e

[2024-01-14 13:39:25,490: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-14 13:39:25,546: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 13:39:25,562: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 64s 6s/step - loss: 8.0182 - accuracy: 0.4820
[2024-01-14 13:40:33,104: INFO: common: json file saved at: scores.json]
{'loss': 8.018155097961426, 'accuracy': 0.48201438784599304}
